# Same Affines, but different CRS!

Here is a case for needing to account for both affine and CRS. Sentinel-2 is delivered on the MGRS grid, which divides UTM Zones into smaller units. Adjacent zones have the same affines, and therefore appear to have the same coordinates. But the CRS differentiates them.

In [1]:
import os 
import xarray as xr 
import pystac
os.environ['VSICURL_PC_URL_SIGNING']='YES'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'

%matplotlib inline

In [2]:
def load_raster(url, asset='B04', overview_level=3, masked=True):
    item = pystac.read_file(url)
    href = item.assets[asset].href
    da = xr.open_dataarray(href, 
                           engine='rasterio', 
                           masked=masked,
                           open_kwargs=dict(overview_level=overview_level),
                           chunks='auto').squeeze()
    return da

In [3]:
# MGRS Tile 10TDS
url = 'https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250414T190931_R056_T10TDS_20250415T001314'
da1 = load_raster(url)
da1

<xarray.DataArray 'band_data' (y: 687, x: 687)> Size: 2MB
dask.array<getitem, shape=(687, 687), dtype=float32, chunksize=(687, 687), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 5kB 4e+05 4.002e+05 ... 5.095e+05 5.097e+05
  * y            (y) float64 5kB 5.2e+06 5.2e+06 5.2e+06 ... 5.09e+06 5.09e+06
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area

In [4]:
# MGRS Tile 11TMM
url = 'https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250413T184919_R113_T11TMM_20250413T224733'
da2 = load_raster(url)

In [5]:
da1.rio.crs == da2.rio.crs

False

In [6]:
print(da1.rio.crs.to_epsg(), da2.rio.crs.to_epsg())

32610 32611


In [7]:
da1.rio.transform() == da2.rio.transform()

True

In [8]:
# This should raise and error b/c not in fact aligned!
da1 + da2

<xarray.DataArray 'band_data' (y: 687, x: 687)> Size: 2MB
dask.array<add, shape=(687, 687), dtype=float32, chunksize=(687, 687), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 5kB 4e+05 4.002e+05 ... 5.095e+05 5.097e+05
  * y            (y) float64 5kB 5.2e+06 5.2e+06 5.2e+06 ... 5.09e+06 5.09e+06
    spatial_ref  int64 8B 0

## Take advantage of xproj CRSIndex

In [9]:
## WIth xproj
#!pip install xproj
import xproj

In [10]:
da1p = da1.proj.assign_crs(da1.rio.crs)
da2p = da1.proj.assign_crs(da2.rio.crs)

In [11]:
da1p + da2p

MergeError: conflicting values/indexes on objects to be combined for coordinate 'init'
first index: CRSIndex
<Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

second index: CRSIndex
<Projected CRS: EPSG:32611>
Name: WGS 84 / UTM zone 11N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 120°W and 114°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - Alberta; British Columbia (BC); Northwest Terr